In [1]:
import os
import nltk
import pandas as pd
import numpy as np
curentdirectory='/content/drive/MyDrive/Distilbert_model/'
os.chdir(curentdirectory)

In [2]:
# !unzip wordnet.zip
# !unzip stopwords.zip
!pip install torch==1.7.1
!pip install transformers==3.5.1

In [3]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
import keras
from tqdm import tqdm
import pickle
from keras.models import Model
import keras.backend as K
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
import itertools
from keras.models import load_model
from sklearn.utils import shuffle
import torch
from transformers import DistilBertTokenizer, TFDistilBertModel, DistilBertConfig
from transformers import TFDistilBertForSequenceClassification
import torch.nn.functional as nnf

In [87]:
DEVICE = "cuda"
MAX_LEN = 64
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 10 
BERT_PATH = "distilbert-base-uncased"
MODEL_PATH = "DistilBert_modelh.h5"
TRAINING_FILE = "Cleaned_Training_Testing_Mapping_Files/train_iris_fullcleaned.xlsx"
TOKENIZER = DistilBertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)

In [88]:
df=pd.read_excel(TRAINING_FILE)
num_classes=len(df.intent1.unique())

MODEL = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=num_classes)
MODEL.compile(loss=loss,optimizer=optimizer, metrics=[metric])
MODEL.load_weights(MODEL_PATH)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_79', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [89]:
import time
import torch.nn as nn
import pandas as pd
tokenizer = TOKENIZER
max_len = MAX_LEN

In [97]:
sent_csv=pd.read_excel('Cleaned_Training_Testing_Mapping_Files/test_iris_fullcleaned.xlsx')
sentences=sent_csv.utterance.values
d= dict([(i,a) for i, a in zip(df.intent1, df.intent)])

In [98]:
maxindx,predictions,maxindx1,maxindx2,maxindx3,maxindx4,predictions1,predictions2,predictions3,predictions4=[],[],[],[],[],[],[],[],[],[]
top_p1,top_p2,top_p3,top_class1,top_class2,top_class3=[],[],[],[],[],[]

In [99]:
for sent in sentences:
    DistilBert_inp=DistilBert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =64,pad_to_max_length = True,return_attention_mask = True)
    input_ids=DistilBert_inp['input_ids']
    attention_masks=DistilBert_inp['attention_mask']
    input_ids=np.asarray(input_ids).reshape(-1,64)
    attention_masks=np.array(attention_masks).reshape(-1,64)
    outputs = MODEL.predict([input_ids,attention_masks],batch_size=32)
    maxidx = outputs[0].argmax(axis=1)
    maxindx.append(int(maxidx))
    print(d[int(maxidx)])

    predictions.append(d[int(maxidx)])
    # outputs = torch.sigmoid(outputs).cpu().detach().numpy()
    print(str(outputs[0]))
    
    prob = nnf.softmax(torch.tensor(outputs[0]),dim=1)
    top_p, top_class = prob.topk(3, dim = 1)
    print("confidence score",top_p)
    print("confidence score1",top_p[0][0])
    
    top_p1.append(float(top_p[0][0]))
    top_p2.append(float(top_p[0][1]))
    top_p3.append(float(top_p[0][2]))
    print("top_class",top_class)
    print("top_class1",top_class[0][0])

    top_class1.append(int(top_class[0][0]))
    top_class2.append(int(top_class[0][1]))
    top_class3.append(int(top_class[0][2]))
    
    predictions1.append(d[int(top_class[0][0])])
    predictions2.append(d[int(top_class[0][1])])
    predictions3.append(d[int(top_class[0][2])])

sent_csv["pred_idx"]=pd.DataFrame(maxindx)
sent_csv["pred_values"]=pd.DataFrame(predictions)
sent_csv["top_p1"]=pd.DataFrame(top_p1)
sent_csv["top_p2"]=pd.DataFrame(top_p2)
sent_csv["top_p3"]=pd.DataFrame(top_p3)
sent_csv["top_class1"]=pd.DataFrame(top_class1)
sent_csv["top_class2"]=pd.DataFrame(top_class2)
sent_csv["top_class3"]=pd.DataFrame(top_class3)
sent_csv["predictions1"]=pd.DataFrame(predictions1)
sent_csv["predictions2"]=pd.DataFrame(predictions2)
sent_csv["predictions3"]=pd.DataFrame(predictions3)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Streaming output truncated to the last 5000 lines.
   -9.341704   -7.808374   -8.080621   -8.576965   -7.697515   -9.845654
  -10.345461   -8.803234   -9.768884   -8.336227   -9.880661   -9.017037
   -8.004364   -7.7770653  -9.078345   -6.791751   -7.647841  -10.320529
   -8.505137  -10.350077   -9.221068  -12.802926   -7.0489936  -7.4712086
  -10.629637  -10.547422  -10.193272  -10.945142  -12.234359   -9.853972
   -9.417394  -10.498208   -8.881156   -9.233872  -12.50091   -11.76863
  -11.197094   -9.887737   -8.89007    -7.9278426  -8.818178   -9.325086
  -11.449501  -11.28251   -10.691041  -13.355374   -9.296482  -10.143817
  -12.194493  -10.562937   -9.910947  -10.545432  -11.804627  -11.519933
  -13.818613   -8.682087   -6.096302  -12.249491   -9.990308   -8.598919
   -9.178431  -10.388466   -8.647567   -8.290596  -11.926433  -10.41889
  -10.770322   -8.627242   -9.566911  -13.400213  -10.519933  -10.11589
   -9.516135  -11.633268  -12.237235   -8.756114   -8.928599  -10.4341135
 

In [100]:
sent_csv.to_csv('IRIS_predictions.csv',index=None)

In [101]:
from sklearn.metrics import accuracy_score

In [102]:
accuracy_score(sent_csv['intent1'],sent_csv['pred_idx'])

0.763459335624284